# Visualize Bi-level strategies 

In [19]:
import numpy as np
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
import plotly.offline as ply
import plotly.graph_objs as go
ply.offline.init_notebook_mode(connected=True)
import plotly.io as pio

We now turn to the example discussed in Ochs2016:
\begin{equation}
	 \min_\theta \frac{1}{2} |x^* - x(\theta)|^2 \quad \text{ s.t. } x(\theta) = \arg \min_{x \geq 0} \frac{\lambda}{2}|\theta x - y |^2 + \frac{1}{2}|x|^2
\end{equation}
The analytical solution of the lower level problem is given by
\begin{equation}
	x(\theta) = \max \left(\frac{\lambda \theta y }{1 + \lambda \theta^2}, 0 \right).
\end{equation}
We find that the lower-level energy is actually $(\lambda \theta^2 +1)$ - strongly convex.

In [2]:
# Set parameters
x_star = 0.75
y = 2.5 # in comparison to y = 2.0 especially interesting
lmb = 1.0

### Define functions

In [3]:
infty = 10
def E(x,theta):
    E_1 = lmb/2*(theta*x-y)**2 + 1/2*x**2
    return  np.where(x < 0, infty, E_1)

def E_1_conj(z, theta):
    m = 1 + lmb*theta**2
    x_max = (z + lmb*theta*y)/m
    return z*x_max - lmb/2*(theta*x_max-y)**2 - 1/2*x_max**2    

def E_2_conj(z,theta):
    return np.where(z > 0, infty, 0.0)
    
def E_conj(z, theta):
    return E_1_conj(-z, theta) + E_2_conj(z, theta)


def l(x,theta):
    return 0.5*(x-x_star)**2

def solve_E(y,theta):
    return np.maximum(lmb*theta*y/(1+lmb*theta**2), 0)


# Variable ranges for plotting and grid search
theta_range = np.arange(-1, 4,0.05)
x_range = np.arange(0, 2, 0.05)
z_range = np.arange(-4,1,0.05)

#Layout
layout = go.Layout(autosize=True,
                   xaxis=dict(type='linear',
                              autorange=True,
                              title='theta'
                              ),
                   yaxis=dict(type='linear',
                              autorange=True,
                              title='x'))

In [4]:
# Sanity check for functions
theta = 0.5
x_min = solve_E(y,theta)
print(E(x_min,theta))

conj_val = []
for z in z_range:
    conj_val.append(E_conj(z,theta)) 
print(min(conj_val))

2.5
-2.499999999999986


### Plot Loss surface and admissible set

The loss surface of the higher level loss $l(x^*,x(\alpha))$ is plotted as contour plot in $(x,\alpha)$. 
The admissible set $x(\alpha)$ is visualized in orange and the optimal point in green.

In [5]:
loss_grid = np.zeros((theta_range.size, x_range.size))
argmin_x = []
loss = []
for i, theta in enumerate(theta_range):
    for j, x in enumerate(x_range):
        loss_grid[i,j] = l(x,theta)
    x_out = solve_E(y, theta)
    argmin_x.append(x_out)
    loss.append(l(x_out,theta))
theta_id = np.argmin(loss)

data = []
data.append(go.Contour(x = list(theta_range), y=list(x_range), z=loss_grid.T, name = 'loss surface',ncontours=30))
data.append(go.Scatter(x = list(theta_range), y=argmin_x, name = 'admissible set'))

data.append(go.Scatter(x = [theta_range[theta_id]], y = [argmin_x[theta_id]], name = 'optimal value'))
ply.offline.iplot(go.Figure(data=data, layout=layout))

### Primal  Surrogate


\begin{align}
	&\min_\theta \max_x \frac{1}{1 + \lambda \theta^2} \left( E(x^*,y,\theta) - E(x,y,\theta) \right) \\
\end{align}

We now visualize the loss surface of this surrogate function as a contour. The new objective is to find a saddle-point of this energy.

In [6]:
def primal_surrogate(x,theta):
    m = 1 + lmb*theta**2
    return 1/m*(E(x_star,theta) - E(x,theta))

surrogate_grid = np.zeros((theta_range.size, x_range.size))
argmin_x = []
loss_direct = []
loss_surr = []
for i, theta in enumerate(theta_range):
    for j, x in enumerate(x_range):
        surrogate_grid[i,j] = primal_surrogate(x,theta)
    x_out = solve_E(y, theta)
    argmin_x.append(x_out)
    loss_direct.append(l(x_out,theta))
    loss_surr.append(primal_surrogate(x_out,theta))
theta_direct_opt = np.argmin(loss_direct)
theta_surr_opt = np.argmin(loss_surr)

data = []
data.append(go.Contour(x = list(theta_range), 
                       y=list(x_range), z=surrogate_grid.T, name = 'loss surface',ncontours=30, showscale=False))
data.append(go.Scatter(x = list(theta_range), y=argmin_x, name = 'admissible set'))
data.append(go.Scatter(x = [theta_range[theta_direct_opt]], 
                       y = [argmin_x[theta_direct_opt]], name = 'optimal value'))
data.append(go.Scatter(x = [theta_range[theta_surr_opt]], 
                       y = [argmin_x[theta_surr_opt]], name = 'optimal value of surrogate'))



ply.offline.iplot(go.Figure(data=data, layout=layout))

We can also visualize the difference between these energies at $x(\alpha)$:

In [7]:
data = []
data.append(go.Scatter(x = theta_range, 
                       y=loss_surr, name = 'Surrogate loss for optimal x(alpha)'))
data.append(go.Scatter(x = [theta_range[theta_surr_opt]], 
                       y= [loss_surr[theta_surr_opt]], name='Optimal Value of Surrogate'))

data.append(go.Scatter(x = [theta_range[theta_direct_opt]], 
                       y= [loss_direct[theta_direct_opt]], name='Optimal Value of original loss'))
data.append(go.Scatter(x = theta_range, 
                       y=loss_direct, name = 'Original loss for optimal x(alpha)'))

layout.yaxis['title'] = 'Energy Value'
fig0 = go.Figure(data=data, layout=layout)
ply.iplot(fig0)

### Dual Surrogate

\begin{equation}
		\min_\theta \min_{z \leq 0} \frac{1}{1 + \lambda \theta^2} \left( E(x^*,y,\theta) + E_1^*(-z,y,\theta) \right) \\
\end{equation}

We have rewritten the problem to a minimization problem and are now looking for the minimizer in $(z,\alpha)$. We visualize the new energy landscape in a log plot.

In [8]:
def dual_surrogate(z,theta):
    m = 1 + lmb*theta**2
    E_temp =  1/m*(E(x_star,theta) + E_conj(z,theta))
    return np.where(z > 0, infty, E_temp)
surrogate_grid = np.zeros((theta_range.size, z_range.size))
argmin_x = []
argmin_z = []
loss_direct = []
loss_surr = []

for i, theta in enumerate(theta_range):
    for j, z in enumerate(z_range):
        surrogate_grid[i,j] = dual_surrogate(z,theta)
    x_out = solve_E(y, theta)
    z_out = z_range[np.argmin(surrogate_grid[i,:])]
    argmin_x.append(x_out)
    argmin_z.append(z_out)
    loss_direct.append(l(x_out,theta))
    loss_surr.append(dual_surrogate(z_out,theta))
theta_direct_opt = np.argmin(loss_direct)
theta_surr_opt = np.argmin(loss_surr)

data = []
data.append(go.Contour(x = list(theta_range), 
                       y=list(z_range), z=surrogate_grid.T, name = 'loss surface',ncontours=30, showscale=False))
data.append(go.Scatter(x = list(theta_range), 
                       y=argmin_z, name = 'minimal z'))
data.append(go.Scatter(x = [theta_range[theta_direct_opt] for x in z_range], 
                       y = list(z_range), name = 'optimal alpha'))
data.append(go.Scatter(x = [theta_range[theta_surr_opt]], 
                       y = [argmin_z[theta_surr_opt]], name = 'optimal value of surrogate'))

layout.yaxis['title'] = 'z'
ply.offline.iplot(go.Figure(data=data, layout=layout))

### Weaker - but simpler surrogates

We now compare to the other two bounds

The two weaker bounds are 
\begin{align}
		&\min_\alpha \min_{q \in \partial E_2(x^*)} D_{E_1^*}(-q,\nabla E_1(x^*)) \\
		=&\min_\alpha \min_{q \in \partial E_2(x^*)} W_{E_1}(-q,x^*) \\
		=&\min_\alpha \min_{q \in \partial E_2(x^*)} E_1(x^*) + E_1^*(-q) + \langle q,x^* \rangle
\end{align}
and
\begin{align}
	&\min_\alpha \min_{q \in \partial E_2(x^*)} D_{E_2^*}(-\nabla E_1(x^*),q) \\
	=&\min_\alpha W_{E_2}(-\nabla E_1(x^*),x^*) \\
	=&\min_\alpha E_2(x^*)  + E_2^*(-\nabla E_1(x^*)) + \langle \nabla E_1(x^*),x^*\rangle 
\end{align}

In [14]:
def grad_E_1(x, theta):
    return lmb*theta*(theta*x-y) + x

def E_1_bound(alpha):
    q = np.where(x_star < 0, infty, 0.0)
    E_1 = lmb/2*(theta*x_star-y)**2 + 1/2*x_star**2
    E_1q = E_1_conj(q, theta)
    return E_1 + E_1q - x_star*q

infty = 2
def E_2_bound(alpha):
    grad = grad_E_1(x_star, theta)
    # E_2 = np.where(x_star < 0, infty, 0.0) + E_2_conj(-grad,theta) + x_star*grad
    infty = 1
    E_2 = np.where(x_star < 0 or -grad > 0, infty, x_star*grad).item()
    return E_2

loss_E_1 = []
loss_E_2 = []
loss_direct = []
loss_surrogate = []
for theta in theta_range:
    loss_E_1.append(E_1_bound(theta))
    loss_E_2.append(E_2_bound(theta))
    
    x_out = solve_E(y,theta)
    loss_direct.append(l(x_out,theta))
    loss_surrogate.append(primal_surrogate(x_out,theta))

In [15]:
data = []
data.append(go.Scatter(x = theta_range, y=loss_surrogate, name = 'Surrogate loss for optimal x(alpha)'))
data.append(go.Scatter(x = theta_range, y=loss_direct, name = 'Original loss for optimal x(alpha)'))
data.append(go.Scatter(x = theta_range, y=loss_E_1, name = 'E_1 bound'))
data.append(go.Scatter(x = theta_range, y=loss_E_2, name = 'E_2 bound'))

data.append(go.Scatter(x = [theta_range[theta_direct_opt]], 
                       y= [loss_direct[theta_direct_opt]], name='Optimal Value of original loss'))

layout.yaxis['title'] = 'Energy Value'
fig0 = go.Figure(data=data, layout=layout)
ply.iplot(fig0)

Note that the flat line in the red curve (E_2 bound) actually marks where this function is infinite. This just visualized lazily.

In [17]:
data = []


data.append(go.Scatter(x = theta_range, y=loss_direct, name = 'Original Loss',   # '$l(x^*,x(ϑ))$   
               line = dict(
                color = ('rgb(22, 96, 167)'),
                width = 4)
                ))
data.append(go.Scatter(x = [theta_range[theta_direct_opt]], 
                       y= [loss_direct[theta_direct_opt]], name= '', #'$l(x^*,x(ϑ)) $   ',
                       line = dict(
                        color = ('rgb(22, 96, 167)'),
                        width = 4),
                        showlegend=False))
data.append(go.Scatter(x = theta_range, y=loss_surrogate, name = 'Bregman Surrogate',#'$D_{E}(x^*,x(ϑ)) $   ',
                       line = dict(
                        #color = ('rgb(22, 96, 167)'),
                        dash = 'dot',
                        width = 4)
                        ))
data.append(go.Scatter(x = theta_range, y=loss_E_1, name = 'Gradient Penalty',#'$W_{E_1}(-z,x^*) $   ', #', z \in \partial E_2(x^*)$',
                       line = dict(
                        #color = ('rgb(22, 96, 167)'),
                        dash = 'dash',
                        width = 4)
                        ))
data.append(go.Scatter(x = theta_range, y=loss_E_2, name = 'Partial Surrogate',#'$W_{E_2}(z,x^*) $   ', #', z \in \partial E_1(x^*)$',
                       line = dict(
                        #color = ('rgb(22, 96, 167)'),
                        dash = 'dash',
                        width = 4)
                        ))



layout.yaxis['title'] = 'Energy Value'
layout.xaxis['title'] = 'ϑ'
layout.xaxis['range'] = [-0.5, 4]
layout.xaxis['autorange'] = False
layout.yaxis['range'] = [0,0.5]
layout.yaxis['autorange'] = False
layout.showlegend = True
layout.legend.x=.8
layout.legend.font.size = 16
layout.legend.borderwidth = 2
layout.margin.pad = 0
fig0 = go.Figure(data=data, layout=layout)
ply.iplot(fig0)

In [20]:
pio.write_image(fig0, 'imgs/Visualization_of_Surrogates_with_names_peters_example.pdf')